In [18]:
import csv
import numpy as np

In [19]:
tweets = []
vocabulary = set()

with open("Preprocessed.csv", encoding="utf8") as file:
    data = csv.reader(file)
    data_list = list(data)

    for tweet in data_list:

        sentence = tweet[2]
        tweets.append(sentence)

print(len(tweets))


4287


In [20]:
bigram_list = []
bigram_count = {}
word_count = {}

for tweet in tweets:

    tokenized_tweet = tweet.split()

    for i in range(len(tokenized_tweet)-1):
        bigram = (tokenized_tweet[i], tokenized_tweet[i+1])
        bigram_list.append(bigram)

        if bigram in bigram_count:
            bigram_count[bigram] += 1
        else:
            bigram_count[bigram] = 1
        
        if tokenized_tweet[i] in word_count:
            word_count[tokenized_tweet[i]] += 1
        else:
            word_count[tokenized_tweet[i]] = 1
    
        vocabulary.add(tokenized_tweet[i])
        
print(len(bigram_list))
print(len(bigram_count))
print(len(word_count))
print(len(vocabulary))

24848
22069
5199
5199


In [31]:
unigram_probability_matrix = {}
total_words_in_vocab = len(vocabulary)

for i in word_count:
    unigram_probability_matrix[i] = word_count[i]/total_words_in_vocab


In [21]:
bigram_probability_matrix = np.zeros((len(vocabulary), len(vocabulary)))

vocabulary_list = list(vocabulary)

for row in range(len(vocabulary_list)):
    for column in range(len(vocabulary_list)):

        current_bigram = (vocabulary_list[row], vocabulary_list[column])
        current_bigram_count = 0

        if current_bigram in bigram_count:
            current_bigram_count = bigram_count[current_bigram]
            
        bigram_probability_matrix[row, column] = (current_bigram_count + 1) / (word_count[vocabulary_list[row]] + len(vocabulary))

In [22]:
# with np.printoptions(threshold=np.inf):
# np.savetxt("Matrix.csv", bigram_probability_matrix, delimiter =",", fmt ='% s')
# print(bigram_probability_matrix)

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math
sentiment_analyzer_object = SentimentIntensityAnalyzer()

# vocab_sentiment = {}
bigram_sentiment = np.zeros((len(vocabulary), len(vocabulary)))

# vocabulary_list = list(vocabulary)
# print(vocabulary_list)

for row in range(len(vocabulary_list)):
    for column in range(len(vocabulary_list)):
        bigram = vocabulary_list[row] + " " + vocabulary_list[column]
        sentiment = sentiment_analyzer_object.polarity_scores(bigram)
        bigram_sentiment[row,column] = 4 + (math.log10(bigram_probability_matrix[row, column])) + (abs(sentiment['compound'])*10)
    
np.savetxt("Matrix.csv", bigram_sentiment, delimiter =",", fmt ='% s')

In [24]:
import random

random_sentence_list = []
# num = 0
while (len(random_sentence_list) != 500):
    
    random_word_list = []
    rand = random.randrange(7,15)
    for i in range(rand):
        random_word_list.append(random.randrange(0, len(vocabulary_list)))
        
    bigram_score = 1
    
    for i in range(len(random_word_list)-1):
        bigram_score *= bigram_sentiment[random_word_list[i], random_word_list[i+1]]
        
    if (bigram_score > pow(0.3,(rand-1))):
        final_sentence = []
        
        for x in random_word_list:
            final_sentence.append(vocabulary_list[x])
        
        sentence = " ".join(final_sentence)
        # sentiment = sentiment_analyzer_object.polarity_scores(sentence)
        random_sentence_list.append(sentence)
        # if(sentiment['compound'] >0.05 or sentiment['compound'] <-0.05):
        #     num+=1
        
# print(random_sentence_list)
# print(num)

In [25]:
generated_samples = []

for i in random_sentence_list:
    sentiment = sentiment_analyzer_object.polarity_scores(i)
    if(sentiment['compound'] > 0):
        generated_samples.append([1,i])
    elif sentiment['compound']<0:
        generated_samples.append([0,i])
        
        
# print(generated_samples)

In [43]:
def perplexity_calculator(sentence):

    final_perplexity = 1

    tokenized_sentence = sentence.split()
    sentence_length = len(tokenized_sentence)

    first_word_base = unigram_probability_matrix[tokenized_sentence[0]]
    first_word_power = -(1/sentence_length)


    final_perplexity *= pow(first_word_base, first_word_power)

    for i in range(sentence_length-1):

        word1 = tokenized_sentence[i]
        word2 = tokenized_sentence[i+1]

        temp_base = (bigram_probability_matrix[(vocabulary_list.index(word1), vocabulary_list.index(word2))])
        temp_power = -(1/sentence_length)

        final_perplexity *= pow(temp_base, temp_power)


    return final_perplexity

perplexity_calculator("good morning world another mother")

719.954288832785

In [44]:
total_generated_perplexity = 0

for i in random_sentence_list:
    total_generated_perplexity += perplexity_calculator(i)

avg_generated_perplexity = total_generated_perplexity/len(random_sentence_list)
print(avg_generated_perplexity)

4898.360927259621
